In [2]:
import tensorflow as tf

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import numpy as np

In [7]:
! /anaconda/envs/py35/bin/python -m pip freeze
! /anaconda/envs/py35/bin/python -m pip --version

absl-py==0.1.10
adal==0.5.0
alabaster==0.7.10
alembic==0.9.8
anaconda-client==1.6.9
anaconda-project==0.8.2
appdirs==1.4.3
applicationinsights==0.11.1
argcomplete==1.9.4
asn1crypto==0.24.0
astor==0.6.2
astroid==1.6.1
astropy==2.0.3
attrs==17.4.0
audioread==2.1.5
autovizwidget==0.12.5
azure==3.0.0
azure-batch==4.0.0
azure-batch-extensions==1.0.1
azure-cli==2.0.22
azure-cli-acr==2.0.15
azure-cli-acs==2.0.19
azure-cli-advisor==0.1.0
azure-cli-appservice==0.1.20
azure-cli-backup==1.0.3
azure-cli-batch==3.1.7
azure-cli-batchai==0.1.3
azure-cli-billing==0.1.6
azure-cli-cdn==0.0.10
azure-cli-cloud==2.0.10
azure-cli-cognitiveservices==0.1.9
azure-cli-command-modules-nspkg==2.0.1
azure-cli-component==2.0.8
azure-cli-configure==2.0.12
azure-cli-consumption==0.1.6
azure-cli-container==0.1.13
azure-cli-core==2.0.22
azure-cli-cosmosdb==0.1.15
azure-cli-dla==0.0.15
azure-cli-dls==0.0.18
azure-cli-eventgrid==0.1.5
azure-cli-extension==0.0.6
azure-cli-feedback==2.0.6
azure-cli-find==0.2.7
azure-cli-in

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
pip 9.0.1 from /anaconda/envs/py35/lib/python3.5/site-packages (python 3.5)


In [9]:
import sys
sys.version

'3.5.4 |Anaconda custom (64-bit)| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [11]:
! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  3425k      0  0:00:09  0:00:09 --:--:-- 6753k


In [69]:
! /anaconda/envs/py35/bin/python -m pip install azure-cognitiveservices-vision-customvision


    100% |████████████████████████████████| 81kB 5.3MB/s ta 0:00:011


### Computer Vision API challenge

In [1]:
import os
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import pickle
import json
from IPython.display import Image, display

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [ ]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

#declaration of important keys for API
prediction_key = "e8f0a200179f4f448594bff294ed85d5"
_url = "https://southcentralus.api.cognitive.microsoft.com/customvision/v1.1/Prediction/7792dd80-3e3d-40d9-b97e-9bd02d9030de/url?iterationId=49453696-73f6-4889-acb1-b1cf9451ac34"
iteration_id= "49453696-73f6-4889-acb1-b1cf9451ac34"
project_id = "7792dd80-3e3d-40d9-b97e-9bd02d9030de"

#Image to process
urlImage= "https://www.switchbacktravel.com/sites/default/files/styles/new_responsive/public/image_fields/field_imgs_inline/Patagonia%20Pluma%20hardshell.jpg?itok=7irEoADx"

#get predictions from trained model
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)
results = predictor.predict_image_url(project_id, iteration_id, url=urlImage)

for prediction in results.predictions:
    print ("\t" + prediction.tag + ": {0:.2f}%".format(prediction.probability * 100))

	hardshell: 100.00%
	insulated: 0.00%


In [ ]:
training_data = {}
for root,dirs,_ in os.walk('gear_images'):
    for d in dirs:
        images = []
        for filename in os.listdir(os.path.join(root, d)):
#             print (os.path.join(root, d,filename))
            image = cv2.imread(os.path.join(root, d,filename))
    
            blank_image = np.zeros((max(image.shape[0],image.shape[1]), max(image.shape[0],image.shape[1]),3), np.uint8)
            blank_image[:,:,:] = (255,255,255)
            
            if (image.shape[0] < image.shape[1]):
                blank_image[0:image.shape[0], :, :] = image
                
            if (image.shape[0] > image.shape[1]):
                blank_image[:, 0:image.shape[1], :] = image
                
            if (image.shape[0] == image.shape[1]):
                blank_image = image
                
            
#             print (image.shape)
            resized = cv2.resize(blank_image, (128,128), interpolation = cv2.INTER_AREA)
#             print (resized.shape)
            images.append(resized)
#             plt.imshow(resized)
#             plt.show()
            
        training_data[d] = images
        print (d)
    
with open('output.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


### START MODEL TRAINING

In [1]:
import os
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import pickle
import json
from IPython.display import Image, display

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [2]:
with open(r"output.pickle", "rb") as input_file:
    data = pickle.load(input_file)
    
labels = []
features = []

for k, v in data.items():
    for value in v:
        features.append(value)
        labels.append(k)
    
labels = np.asarray(labels)
features = np.asarray(features)
print (features.shape)
print (labels.shape)


(2122, 128, 128, 3)
(2122,)


In [3]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score



In [7]:
def train_classifier(clf, X_train, y_train):

    ''' Fits a classifier to the training data. '''

   

    # Start the clock, train the classifier, then stop the clock


    clf.fit(X_train, y_train)

 

    #return training time
    return clf
 

   

def predict_labels(clf, train, target):

    ''' Makes predictions using a fit classifier based on F1 score. '''

   

    # Start the clock, make predictions, then stop the clock

    y_pred = clf.predict(train)

 

    #cnf_matrix = confusion_matrix(target, y_pred)

#     plot_confusion_matrix(cnf_matrix,classes = y_train.values

#                       ,title='Confusion matrix')

   

    # Print and return results

    #returns prediction time and f1 score

    return (accuracy_score(target, y_pred))

 

def train_predict(clf, X_train, y_train, X_test, y_test):

    ''' Train and predict using a classifer based on F1 score. '''

   

    # Train the classifier

    clf = train_classifier(clf, X_train, y_train)

   

    # results of prediction for training

    (train_score) = predict_labels(clf, X_train, y_train)

 

    #results of prediction for testing

    ( test_score) = predict_labels(clf, X_test, y_test)

 

    #return (tranining time, training score, prediction time, testing score)

    return (train_score, test_score)

 

 

def plot_confusion_matrix(cm, classes,

                          normalize=False,

                          title='Confusion matrix',

                          cmap=plt.cm.Blues):

    """

    This function prints and plots the confusion matrix.

    Normalization can be applied by setting `normalize=True`.

    """

    if normalize:

        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        print("Normalized confusion matrix")

    else:

        print('Confusion matrix, without normalization')

 

    print(cm)

 

    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.title(title)

    plt.colorbar()

    tick_marks = np.arange(len(classes))

    plt.xticks(tick_marks, classes, rotation=45)

    plt.yticks(tick_marks, classes)

 

    fmt = '.2f' if normalize else 'd'

    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):

        plt.text(j, i, format(cm[i, j], fmt),

                 horizontalalignment="center",

                 color="white" if cm[i, j] > thresh else "black")

 

    plt.tight_layout()

    plt.ylabel('True label')

    plt.xlabel('Predicted label')

 

 



In [19]:
from sklearn.ensemble import RandomForestClassifier
import requests
from io import BytesIO
from PIL import Image

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# print (X_train.shape)

clf = RandomForestClassifier()
(a,b) = train_predict(clf, X_train, y_train, X_test, y_test)

clf = clf.fit(X_train, y_train)

urlImage= "https://shop.epictv.com/sites/default/files/ae42ad29e70ba8ce6b67d3bdb6ab5c6e.jpeg"
#preprocess URL Image
response = requests.get(urlImage)
img = np.asarray(Image.open(BytesIO(response.content)))
resized = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
resized = resized.reshape(1, -1)
print (resized.shape)
prediction = clf.predict(resized)

print (prediction)
print (clf.predict_proba(resized))
print (np.unique(y_train))
print (b)


(1, 49152)
['helmets']
[[0.  0.  0.  0.  0.  0.  0.1 0.7 0.  0.  0.1 0.1]]
['axes' 'boots' 'carabiners' 'crampons' 'gloves' 'hardshell_jackets'
 'harnesses' 'helmets' 'insulated_jackets' 'pulleys' 'rope' 'tents']
0.8094117647058824
